In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [16]:
data = pd.read_csv("./data/Datasets_filtering_2.csv")

In [17]:
text_data = data['description'].tolist()

In [25]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
max_len = max(len(seq) for seq in sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

In [26]:
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

In [61]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(max_len)
])

In [62]:
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr = 0.0001)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics = ['accuracy'])

In [64]:
model.fit(padded_sequences, padded_sequences, epochs=1000)

Epoch 1/1000
9/9 [==============================] - 1s 43ms/step - loss: 173818.4688 - accuracy: 0.0440
Epoch 2/1000
9/9 [==============================] - 0s 44ms/step - loss: 173426.8438 - accuracy: 0.0513
Epoch 3/1000
9/9 [==============================] - 0s 42ms/step - loss: 170192.6250 - accuracy: 0.0513
Epoch 4/1000
9/9 [==============================] - 0s 42ms/step - loss: 154621.3281 - accuracy: 0.0513
Epoch 5/1000
9/9 [==============================] - 0s 44ms/step - loss: 140543.1406 - accuracy: 0.0513
Epoch 6/1000
9/9 [==============================] - 0s 41ms/step - loss: 133777.0156 - accuracy: 0.0513
Epoch 7/1000
9/9 [==============================] - 0s 44ms/step - loss: 131509.1406 - accuracy: 0.0366
Epoch 8/1000
9/9 [==============================] - 0s 45ms/step - loss: 129755.1094 - accuracy: 0.1209
Epoch 9/1000
9/9 [==============================] - 0s 43ms/step - loss: 128634.4609 - accuracy: 0.1941
Epoch 10/1000
9/9 [==============================] - 0s 43ms/ste

In [65]:
embeddings = model.predict(padded_sequences)

9/9 [==============================] - 0s 10ms/step


In [66]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [73]:
user_input = "dingin dan segar"
user_sequence = tokenizer.texts_to_sequences([user_input])
user_padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(user_sequence, maxlen=padded_sequences.shape[1])

In [74]:
user_embedding = model.predict(user_padded_sequence)[0]

1/1 [==============================] - 0s 24ms/step


In [75]:
similarity_scores = []
for embedding in embeddings:
    similarity_scores.append(cosine_similarity(user_embedding, embedding))

In [76]:
top_records_indices = np.argsort(similarity_scores)[-5:]
top_records = data.iloc[top_records_indices]

In [77]:
print("Most similar data records:")
print(top_records)

Most similar data records:
     _id      Kota                                nama  \
166  167    Malang                     Hawai Waterpark   
39    40    Blitar                   Candi Sumbernanas   
8      9    Blitar  Wisata Pantai Umbul Waru di Blitar   
228  229  Surabaya                         Bento Kopi    
47    48    Blitar   Kolam Renang Tirto Agung Tambakan   

                                           description  
166  Nikmati sensasi keseruan dan kesegarannya untu...  
39   Candi Sumbernanas merupakan sebuah candi yang ...  
8    Tempat wisata di Blitar yang lagi hits adalah ...  
228  Tempat nongkrong yang menjual berbagai minuman...  
47   kolam renang tirto agung tambakan di desa gand...  
